# Voyages API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VoyagesAPI/VoyagesAPI.ipynb). 

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [21]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the Voyages API
The Voyages API retrieves information about vessel voyages.

In [22]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
from signal_ocean.voyages import Vessel, VesselFilter
from signal_ocean.voyages import VesselType, VesselTypeFilter
from signal_ocean.voyages import VesselClass, VesselClassFilter
import pandas as pd
from datetime import date, timedelta

In [23]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages for vessel

In [2]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Sea'))[0]
vessel

Vessel(imo=1014022, vessel_name='Seaexplorer')

In [3]:
vessel_voyages = api.get_voyages(imo=vessel.imo)

In [4]:
vessel_voyages_df = pd.DataFrame([v.__dict__ for v in vessel_voyages])
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing,torres_strait_crossing,magellan_strait_crossing,great_belt_crossing
0,1014022,1,1,88,1,1684,False,"(VoyageEvent(id='IF790658SEDE9BB300', voyage_i...",IF790658VEDE9BB300,1,...,None,1335.66,NaN,None,None,None,None,None,None,None
1,1014022,2,1,88,1,1684,False,"(VoyageEvent(id='IF790658SEDF125A00', voyage_i...",IF790658VEDF125A00,1,...,None,13548.53,NaN,None,None,None,None,None,None,None
2,1014022,3,1,88,1,1684,False,"(VoyageEvent(id='IF790658SEDF617400', voyage_i...",IF790658VEDF617400,1,...,None,4372.47,NaN,None,None,None,None,None,None,None
3,1014022,4,1,88,1,1684,False,"(VoyageEvent(id='IF790658SEDF890100', voyage_i...",IF790658VEDF890100,1,...,None,6217.78,NaN,None,None,None,None,None,None,None
4,1014022,5,1,88,1,1684,False,"(VoyageEvent(id='IF790658SEDFB08E00', voyage_i...",IF790658VEDFB08E00,2,...,None,2756.33,2052.31,None,None,None,None,None,None,None


In [5]:
vessel_events_df = pd.DataFrame(e.__dict__ for voyage_events in vessel_voyages_df['events'] for e in voyage_events)
vessel_events_df.tail(5)

,id,voyage_id,event_type_id,event_type,event_horizon_id,event_horizon,purpose,event_date,arrival_date,sailing_date,...,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,quantity,quantity_unit_id,quantity_unit,quantity_in_barrels,event_details
28,IF790658TEDFCAEC00,None,1,PortCall,0,Historical,Discharge,NaT,2025-06-01 15:51:15+00:00,2025-06-03 15:58:21+00:00,...,25019,Atlantic America,25028,West,None,27000.0,1.0,MetricTonnes,230769.0,"(VoyageEventDetail(id='IF790658LEDFCAEC00', ev..."
29,IF790658SEDFB08E00,None,2,VoyageStart,0,Historical,Start,2025-06-03 15:58:21+00:00,NaT,NaT,...,25019,Atlantic America,25028,West,None,NaN,NaN,None,NaN,None
30,IF790658TEDFD03200,None,0,Stop,0,Historical,Stop,NaT,2025-06-03 23:51:28+00:00,2025-06-04 11:54:36+00:00,...,25019,Atlantic America,25028,West,None,NaN,NaN,None,NaN,"(VoyageEventDetail(id='IF790658LEDFD03200', ev..."
31,IF790658TEDFD81B00,None,1,PortCall,0,Historical,Load,NaT,2025-06-12 23:51:02+00:00,2025-06-15 11:50:03+00:00,...,25019,Atlantic America,25028,West,None,38000.0,1.0,MetricTonnes,316667.0,"(VoyageEventDetail(id='IF790658LEDFD81B00', ev..."
32,IF790658TEDFF27900,None,1,PortCall,2,Future,Discharge,NaT,2025-07-01 14:43:13.321000+00:00,2025-07-04 07:14:19.243000+00:00,...,25025,Mediterranean / UK Continent,25028,West,True,38000.0,1.0,MetricTonnes,316667.0,None


In [6]:
vessel_event_details_df = pd.DataFrame(e.__dict__ for event_details in vessel_events_df['event_details'] for e in event_details or [])
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,sts_id,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name,floating_storage_start_date,floating_storage_duration
26,IF790658LEDFB08E00,None,Stop,2025-05-15 15:48:42+00:00,2025-05-16 07:53:28+00:00,NaT,NaT,None,4427,Lome Lightering Zone,6.0568,1.2939,NaN,None,None,None
27,IF790658LEDFBDBD00,None,Stop,2025-05-22 15:51:57+00:00,2025-05-23 07:53:58+00:00,NaT,NaT,None,5669,Estaçao Da Shell-Gale,14.1189,-26.3622,NaN,None,None,None
28,IF790658LEDFCAEC00,None,Jetty,2025-06-01 15:51:15+00:00,2025-06-03 15:58:21+00:00,2025-06-02 11:57:40+00:00,2025-06-03 15:58:21+00:00,None,5544,New York Kinder Morgan Carteret Terminal,40.5890,-74.2082,NaN,None,None,None
29,IF790658LEDFD03200,None,Stop,2025-06-03 23:51:28+00:00,2025-06-04 11:54:36+00:00,NaT,NaT,None,5544,New York Kinder Morgan Carteret Terminal,40.6419,-74.0631,NaN,None,None,None
30,IF790658LEDFD81B00,None,Jetty,2025-06-12 23:51:02+00:00,2025-06-15 11:50:03+00:00,2025-06-13 23:49:47+00:00,2025-06-15 11:50:03+00:00,None,4716,Marathon Texas City Refinery,29.3736,-94.8893,NaN,None,None,None


### Get voyages for vessel class

In [7]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id
vlcc_id

84

In [8]:
date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [9]:
recent_vlcc_voyages = pd.DataFrame([v.__dict__ for v in recent_vlcc_voyages])
recent_vlcc_voyages.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing,torres_strait_crossing,magellan_strait_crossing,great_belt_crossing
392,9933652,15,1,84,1,558.0,False,"(VoyageEvent(id='I97935454SEDFD81B00', voyage_...",I97935454VEDFD81B00,2,...,863.56,NaN,NaN,None,None,None,None,None,None,None
393,9937799,7,1,84,1,1031.0,False,"(VoyageEvent(id='I97A38754SEDFD81B00', voyage_...",I97A38754VEDFD81B00,3,...,251.92,NaN,NaN,None,None,None,None,None,None,None
394,9941673,15,1,84,1,2236.0,False,"(VoyageEvent(id='I97B2A954SEDFD81B00', voyage_...",I97B2A954VEDFD81B00,2,...,571.16,NaN,NaN,None,None,None,None,None,None,None
395,9943748,11,1,84,1,558.0,False,"(VoyageEvent(id='I97BAC454SEDFB08E00', voyage_...",I97BAC454VEDFB08E00,2,...,NaN,5.7,5759.96,None,None,None,None,None,None,None
396,9988695,1,1,84,1,NaN,False,"(VoyageEvent(id='I986A5754SEDFD81B00', voyage_...",I986A5754VEDFD81B00,2,...,NaN,NaN,NaN,None,None,None,None,None,None,None


### Get voyages for vessel in flat format

In [10]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Sea'))[0]
vessel

Vessel(imo=1014022, vessel_name='Seaexplorer')

In [11]:
vessel_voyages_flat = api.get_voyages_flat(imo=vessel.imo)

In [12]:
vessel_voyages_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.voyages)
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing,torres_strait_crossing,magellan_strait_crossing,great_belt_crossing
0,1014022,1,1,88,1,1684,False,None,IF790658VEDE9BB300,1,...,None,1335.66,NaN,None,None,None,None,None,None,None
1,1014022,2,1,88,1,1684,False,None,IF790658VEDF125A00,1,...,None,13548.53,NaN,None,None,None,None,None,None,None
2,1014022,3,1,88,1,1684,False,None,IF790658VEDF617400,1,...,None,4372.47,NaN,None,None,None,None,None,None,None
3,1014022,4,1,88,1,1684,False,None,IF790658VEDF890100,1,...,None,6217.78,NaN,None,None,None,None,None,None,None
4,1014022,5,1,88,1,1684,False,None,IF790658VEDFB08E00,2,...,None,2756.33,2052.31,None,None,None,None,None,None,None


In [13]:
vessel_events_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.events)
vessel_events_df.tail(5)

,id,voyage_id,event_type_id,event_type,event_horizon_id,event_horizon,purpose,event_date,arrival_date,sailing_date,...,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,quantity,quantity_unit_id,quantity_unit,quantity_in_barrels,event_details
28,IF790658TEDFCAEC00,IF790658VEDF890100,1,PortCall,0,Historical,Discharge,NaT,2025-06-01 15:51:15+00:00,2025-06-03 15:58:21+00:00,...,None,None,None,None,None,27000.0,1.0,MetricTonnes,230769.0,None
29,IF790658SEDFB08E00,IF790658VEDFB08E00,2,VoyageStart,0,Historical,Start,2025-06-03 15:58:21+00:00,NaT,NaT,...,None,None,None,None,None,NaN,NaN,None,NaN,None
30,IF790658TEDFD03200,IF790658VEDFB08E00,0,Stop,0,Historical,Stop,NaT,2025-06-03 23:51:28+00:00,2025-06-04 11:54:36+00:00,...,None,None,None,None,None,NaN,NaN,None,NaN,None
31,IF790658TEDFD81B00,IF790658VEDFB08E00,1,PortCall,0,Historical,Load,NaT,2025-06-12 23:51:02+00:00,2025-06-15 11:50:03+00:00,...,None,None,None,None,None,38000.0,1.0,MetricTonnes,316667.0,None
32,IF790658TEDFF27900,IF790658VEDFB08E00,1,PortCall,2,Future,Discharge,NaT,2025-07-01 14:43:13.321000+00:00,2025-07-04 07:14:19.243000+00:00,...,None,None,None,None,True,38000.0,1.0,MetricTonnes,316667.0,None


In [14]:
vessel_event_details_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.event_details)
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,sts_id,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name,floating_storage_start_date,floating_storage_duration
26,IF790658LEDFB08E00,IF790658TEDFB08E00,Stop,2025-05-15 15:48:42+00:00,2025-05-16 07:53:28+00:00,NaT,NaT,None,4427,Lome Lightering Zone,6.0568,1.2939,NaN,None,None,None
27,IF790658LEDFBDBD00,IF790658TEDFBDBD00,Stop,2025-05-22 15:51:57+00:00,2025-05-23 07:53:58+00:00,NaT,NaT,None,5669,Estaçao Da Shell-Gale,14.1189,-26.3622,NaN,None,None,None
28,IF790658LEDFCAEC00,IF790658TEDFCAEC00,Jetty,2025-06-01 15:51:15+00:00,2025-06-03 15:58:21+00:00,2025-06-02 11:57:40+00:00,2025-06-03 15:58:21+00:00,None,5544,New York Kinder Morgan Carteret Terminal,40.5890,-74.2082,NaN,None,None,None
29,IF790658LEDFD03200,IF790658TEDFD03200,Stop,2025-06-03 23:51:28+00:00,2025-06-04 11:54:36+00:00,NaT,NaT,None,5544,New York Kinder Morgan Carteret Terminal,40.6419,-74.0631,NaN,None,None,None
30,IF790658LEDFD81B00,IF790658TEDFD81B00,Jetty,2025-06-12 23:51:02+00:00,2025-06-15 11:50:03+00:00,2025-06-13 23:49:47+00:00,2025-06-15 11:50:03+00:00,None,4716,Marathon Texas City Refinery,29.3736,-94.8893,NaN,None,None,None


In [15]:
vessel_voyages_geos_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.geos)
vessel_voyages_geos_df.tail(5)

,id,name,port_id,port_unlocode,port_name,country_id,country,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3
46,3684,Port Harcourt,3684,NG PHC,Port Harcourt,171,Nigeria,24772,Africa Atlantic Coast,37,West Africa,25018,Africa,25027,Africa
47,3334,Cape Verde,3334,CV GRA,Cape Verde,59,Cabo Verde,24772,Africa Atlantic Coast,37,West Africa,25018,Africa,25027,Africa
48,3886,Texas City,3886,US TXT,Texas City,240,United States,24676,US Gulf,33,US Gulf & Mainland,25019,Atlantic America,25028,West
49,6328,Petrotitan Xiao Hu Terminal,3289,CN DGG,Dongguan,55,China,24725,South China,17,China / Taiwan,99,Far East,84,East
50,5744,Euro Tank Terminal B.v.,3689,NL RTM,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West


### Get voyages for vessel class in the flat format

In [16]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id

date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages_flat = api.get_voyages_flat(vessel_class_id=vlcc_id, date_from=date_from)

In [17]:
print('Voyages:', len(recent_vlcc_voyages_flat.voyages))
print('Events:', len(recent_vlcc_voyages_flat.events))
print('EventDetails:', len(recent_vlcc_voyages_flat.event_details))
print('Geos:', len(recent_vlcc_voyages_flat.geos))

Voyages: 397
Events: 1443
EventDetails: 534
Geos: 298


### Get voyages for vessel incrementally 

Initial requests retrieves voyages mathcing the query criteria and a query token that can be used in the subsequent request.

In [18]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('XIN YONG YANG'))[0]
vessel

Vessel(imo=9416642, vessel_name='Xin Yong Yang')

In [19]:
vessel_voyages, next_request_token = api.get_incremental_voyages(imo=vessel.imo)
len(vessel_voyages)

92

Provided the token retrieved for the previous request, the subsequent request retrieves only voyages that have been updated since the previous request. 

In [20]:
incremental_voyages, next_request_token = api.get_incremental_voyages(imo=9416642, incremental_token=next_request_token)
len(incremental_voyages)

1

The retrieved information is used to replace any modify voyages from the dataset. Note that returned voyages marked as deleted are only used to filter out voyages. 

In [21]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
vessel_voyages = [v for v in vessel_voyages if v.id not in updated_voyage_ids]
new_vessel_voyages = [v for v in incremental_voyages if not v.deleted]
vessel_voyages = sorted(vessel_voyages + new_vessel_voyages, key= lambda v: v.voyage_number)
len(vessel_voyages)

92

### Get voyages for vessel class incrementally 

Voyages for vessel class are retieved and updated in the same way incrementally.

In [22]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

84

In [23]:
date_from = date.today() - timedelta(days=30)

In [24]:
voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from)
len(voyages)

397

In [25]:
incremental_voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)
len(incremental_voyages)

54

In [26]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids and not v.deleted]
new_voyages = [v for v in incremental_voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

397

### Get voyages for vessel class incrementally in the flat format

Voyages may be retrieved and updated incrementally in the flat format. 

In [27]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

84

In [28]:
date_from = date.today() - timedelta(days=30)

In [29]:
voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from)

In [30]:
voyages = voyages_flat.voyages
events = voyages_flat.events
event_details = voyages_flat.event_details
geos = voyages_flat.geos

In [31]:
len(voyages), len(events), len(event_details), len(geos)

(397, 1443, 534, 298)

In [32]:
incremental_voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)

In [33]:
len(incremental_voyages_flat.voyages)

72

In this case the update step is applied to voyages, events, event details and geos datasets.

In [34]:
updated_voyage_ids = set(v.id for v in incremental_voyages_flat.voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids and not v.deleted]
new_voyages = [v for v in incremental_voyages_flat.voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

397

In [35]:
updated_event_ids = set(e.id for e in events if e.voyage_id in updated_voyage_ids)
events = [e for e in events if e.id not in updated_event_ids]
events = sorted(events + list(incremental_voyages_flat.events), key= lambda e: e.id)
len(events)

1443

In [36]:
event_details = [e for e in event_details if e.id not in updated_event_ids]
event_details = sorted(event_details + list(incremental_voyages_flat.event_details), key= lambda e: e.id)
len(event_details)

619

In [37]:
updated_geo_ids = set(g.id for g in incremental_voyages_flat.geos)
geos = [g for g in geos if g.id not in updated_geo_ids] + list(incremental_voyages_flat.geos)
len(geos)

298